In [61]:
import pandas as pd
from tqdm import tqdm
import numpy as np
from scipy import stats
from statsmodels.stats.meta_analysis import effectsize_smd
from statsmodels.stats import proportion
from statsmodels.stats.power import tt_ind_solve_power
from statsmodels.stats.power import zt_ind_solve_power

In [62]:
data = pd.read_csv('data/gb_sem_8_cw.csv')

In [63]:
df = pd.read_excel('data/gb_sem_8_hm.xlsx')

d:\Mytraining\.venv.AB.Testing\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning:

Unknown extension is not supported and will be removed



In [64]:
df.head()

,USER_ID,VARIANT_NAME,REVENUE
0,737,variant,0.0
1,2423,control,0.0
2,9411,control,0.0
3,7311,control,0.0
4,6174,variant,0.0


In [65]:
df.nunique()

USER_ID         6324
VARIANT_NAME       2
REVENUE          101
dtype: int64

In [66]:
df = df.groupby(['USER_ID', 'VARIANT_NAME'], as_index=False).agg({'REVENUE': 'sum'})
df

,USER_ID,VARIANT_NAME,REVENUE
0,2,control,0.00
1,3,control,0.00
2,3,variant,0.00
3,4,variant,0.00
4,5,variant,0.00
...,...,...,...
7860,9996,control,0.00
7861,9996,variant,6.46
7862,9998,control,0.00
7863,10000,control,0.00


In [67]:
df.groupby('USER_ID', as_index=False).agg({'VARIANT_NAME': 'count'})['VARIANT_NAME'].value_counts()

VARIANT_NAME
1    4783
2    1541
Name: count, dtype: int64

In [68]:
unique_ids = \
(df
 .groupby('USER_ID', as_index=False)
 .agg({'VARIANT_NAME': 'count'})
#  .['VARIANT_NAME'].value_counts()
 .query('VARIANT_NAME == 1')
 .USER_ID
 .values
 )

In [69]:
df_new = df[df.USER_ID.isin(unique_ids)].copy(deep=True)
df

,USER_ID,VARIANT_NAME,REVENUE
0,2,control,0.00
1,3,control,0.00
2,3,variant,0.00
3,4,variant,0.00
4,5,variant,0.00
...,...,...,...
7860,9996,control,0.00
7861,9996,variant,6.46
7862,9998,control,0.00
7863,10000,control,0.00


In [70]:
df_new.describe()


,USER_ID,REVENUE
count,4783.000000,4783.000000
mean,4994.395777,0.135873
std,2898.618472,3.011392
min,2.000000,0.000000
25%,2476.000000,0.000000
50%,4975.000000,0.000000
75%,7515.000000,0.000000
max,9998.000000,196.010000


In [71]:
control = data[data['ab_group'] == 'A'].copy(deep=True)
treatment = data[data['ab_group'] == 'B'].copy(deep=True)

In [72]:
def continious_result(control: pd.DataFrame,
                      treatment: pd.DataFrame,
                      column: str,
                      n_iters: int = 10_000) -> pd.DataFrame:
    # Статистика по выборкам
    size = control.loc[:, column].shape[0]
    
    control_mean = control.loc[:, column].mean()
    treatment_mean = treatment.loc[:, column].mean()
    
    control_std = control.loc[:, column].std(ddof=1)
    treatment_std = treatment.loc[:, column].std(ddof=1)
    
    # Бутсрап
    booted_diff = []
    for _ in tqdm(range(n_iters)):
        control_sample = control.loc[:, column].sample(n=size, replace=True).values
        treatment_sample = treatment.loc[:, column].sample(n=size, replace=True).values
        booted_diff.append(np.mean(control_sample - treatment_sample))
    
    # Считаем статистику после бустрапа
    md_ci, std_ci = np.mean(booted_diff), np.std(booted_diff, ddof=1)
    left_ci, right_ci = np.percentile(booted_diff, [2.5, 97.5])
    p_value_ci = 2 * (1 - stats.norm.cdf(np.abs(md_ci / std_ci)))
    
    # Считаем мощность эксперимента
    effect_size, _ = effectsize_smd(mean1=treatment_mean, sd1=treatment_std, nobs1=size,
                                    mean2=control_mean, sd2=control_std, nobs2=size)
    power = tt_ind_solve_power(effect_size=effect_size,
                               nobs1=size,
                               alpha=.05,
                               power=None,
                               ratio=1)
    # Формируем отчёт 
    result = pd.DataFrame({'effect_size': effect_size,
                           'alpha': p_value_ci, 
                           'beta': (1-power),
                           'CI': f'[{np.round(left_ci, 3)}, {np.round(right_ci, 3)}]',
                           'difference': md_ci,},
                          index=[column]) 
    return result

In [73]:
import plotly.express as px
fig = px.histogram(data,
                   x='revenue',
                   color = 'ab_group',
                   title='REVENUE',
                   marginal = 'box',
                   nbins = 100,
                   barmode='overlay')

fig.show()


In [74]:
continious_result(control, treatment, column='revenue')

100%|██████████| 10000/10000 [00:13<00:00, 726.78it/s]


,effect_size,alpha,beta,CI,difference
revenue,-0.016711,0.197894,0.749424,"[-0.032, 0.434]",0.159111


**Вывод:** После проведения экспемента получаем, что alpha и beta больше своих номинальных значений. А также доверительный интервал проходит через 0. Делаем вывод что статичкески значимых различий по revrnue между группами нет.